# Hurst Exponent

This implementation of R/S Analysis to get the Hurst exponent, is inspired in different sources.

Sources:
* **Fractal Market Analysis**, Edgar Peters (1994), capitulo 4.
* [Python's Hurst library](https://github.com/Mottl/hurst)
* [**A brief history of long memory: Hurst, Mandelbrot and the road to ARFIMA**](https://arxiv.org/abs/1406.6018v3), Graves, T. & Gramacy, R. & Watkins, N. & Franzke, C.

<font color='red'>NOT FINISHED YET! .. function is not working properly</font> 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from hurst import compute_Hc, random_walk

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import math
from scipy import optimize

In [ ]:
np.random.seed(200)

# Persistent Values

In [ ]:
testing_data = {
    'example_1' : np.array(random_walk(5000, proba=0.5))+500, # BM
    'example_2' : np.array(random_walk(1000, proba=0.9))+10000, # Persistent
    'example_3' : np.array(random_walk(1000, proba=0.2))+10000, # Anti-Persistent
    'example_4' : np.array([i + np.random.rand()*50 for i in range(300)]), # Persistent
    'example_5' : np.array([2+math.sin(i)+np.random.rand()*1.000001 for i in np.linspace(0,40,300)]) # Anti-Persistent
}

In [ ]:
ts = testing_data['example_2']

N = len(ts)
X_k = np.log(ts[1:] / ts[:-1]) # See Peters (1994) <- it doesn't work with negative differences
# X_k = ts[1:] - ts[:-1]
windows_ = [len(ts)]
while windows_[-1] > 5:
    windows_.append(math.floor(windows_[-1] / 2 ))

In [ ]:
plt.plot(ts)
plt.show()

In [ ]:
RS_values = []
n_values = []
# windows_ = [10, 17, 31, 56, 100]
for w_size in windows_:
    RS_values_I = []
    for i in range(0,N, w_size):
        if i+w_size > N:
            continue
        subset_ = X_k[i:i+w_size]
        X_mean = np.mean(subset_)
        X_z = subset_ - X_mean
        Y_k = np.cumsum(X_z)
        R_n = max(Y_k) - min(Y_k)
        S_n = np.std(subset_, ddof=0)
        RS_values_I.append(R_n / S_n)
    RS_values.append(np.mean(RS_values_I))
    n_values.append(w_size)
    del RS_values_I
model = linregress(np.log(n_values), np.log(RS_values))
print(f"=======\n{model.slope=:.5}\n{model.intercept=:.5}")
print(f"=======\n HURST  = {model.slope=:.5}")

In [ ]:
# list(zip(RS_values, n_values))

In [ ]:
plt.plot(np.log(n_values), np.log(RS_values))
plt.show()

In [ ]:
H, c, data = compute_Hc(ts, kind='price', simplified=True)
print("Price :: H={:.4f}, c={:.4f}".format(H,c))

In [ ]:
H, c, data = compute_Hc(ts, kind='random_walk', simplified=True)
print("Random Walk     :: H={:.4f}, c={:.4f}".format(H,c))

In [ ]:
H, c, data = compute_Hc(ts, kind='change', simplified=True)
print("Change     :: H={:.4f}, c={:.4f}".format(H,c))